In [35]:
import csv
from collections import defaultdict
import numpy as np

In [2]:
uKeys = ['username', 'userId', 'name', 'tags', 'followers', 'following', 'bio', 'followerCount', 'followCount', 'postCount', 'posts', 'recommends']
pKeys = ["id", "clapCount", "wordCount", "readingTime", "tags", "title", "userId", "time", "subtitle"]

In [93]:
def readCSV(filename, keys):
    objects = []
    
    with open(filename) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        
        for row in csv_reader:
            if line_count == -1:
                print(f'Column names are {", ".join(row)}')
                line_count += 1
            else:

                u = {}
                for i,k in enumerate(keys): 
                    u[k] = row[i]
                objects.append(u)

                line_count += 1
                
    print(f'Processed {line_count} lines.')
    return objects

In [231]:
readUsers = readCSV('users.csv', uKeys)
users = defaultdict(dict)

for u in readUsers:
    users[u['userId']] = u

Processed 1818 lines.


In [232]:
readPosts = readCSV('posts.csv', pKeys)
posts = defaultdict(dict)

for p in readPosts:
    posts[p['id']] = p

Processed 11013 lines.


In [233]:
len(users)

1697

In [234]:
users['de9100e3b468']

{'username': 'SeattleTallPopp',
 'userId': 'de9100e3b468',
 'name': 'Traca Savadogo',
 'tags': "['Short Story', 'Depression', 'Life Hacking', 'Conversations', 'Overcoming Fear']",
 'followers': "['a201e7e89d55', '73bd8b3a2d14', 'a8f81b56e2d9', '97e183b9e5ec', 'af995ad283f8', '513fd8c6c48e', '5812ff315ba3', '1077f21b7d34', 'fdc9a8b3c06f', '87501ad4ce18', '8cf58e614c41', '492da2345a52', '6d336872b7d9', '66b94c13688', 'fe9bd98a96fc', '131725340eab', 'fb35326084b2', 'a093a1121820', 'a676299cb0d4', 'ef91d328059d', '5c6178e1be98', '13700506390b', '425d307aae41', 'b56852cccf1', 'dfc6a7f537a8', '8ac7d9f012c6', 'e2a24b379abd', 'c0e801d899d4', '9797a289722b', '1bec38279cb0', '22402a42e5c3', '45b5127eec93', '538168dbda3f', '37d215005b17', 'f340cd074f20', '7e2637cf8663', '786474d05f7e', 'c8b51be9b956', 'ff5d07907eaf', '88bf32fc7e88', '9ca00b394617', 'ecb9fc83434d', '4c1d4fb8a6da', 'a2ecbb1eb6e2', '3a4c4f3a2df3', '868cc4f29eb8', 'f599241f2a47', '1dabd7e82ccc', '756992f84111', '27d42380859a', 'eefb1

In [72]:
posts['f12baa81d5c6']

{'id': 'f12baa81d5c6',
 'clapCount': '32',
 'wordCount': '2499',
 'readingTime': '10.263522012578617',
 'tags': "['Public Speaking', 'Speaking', 'Comfort Zone', 'Mentorship', 'Ignite']",
 'title': 'Growth Happens Outside Our Comfort Zone: How I Prepared to Speak in Front of 500+ People & Why I’m…',
 'userId': 'b2d986206d08',
 'time': '1475463567034',
 'subtitle': 'A few months ago I stood on stage at Town Hall Seattle and delivered a five-minute talk at IGNITE Seattle #30 to more than 500 people.'}

In [209]:
testUsers = list(users.keys())
testPosts_ = [ast.literal_eval(users[u]['recommends']) for u in testUsers]
testPosts = []
for arr in testPosts_:
    for p in arr: testPosts.append(p)

In [210]:
import random
testPosts = list(set(testPosts))
random.shuffle(testPosts)
len(testPosts)

4286

In [235]:
matrix = np.zeros((len(testUsers), len(testPosts)))
matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [240]:
for i,u in enumerate(testUsers):
    for j,p in enumerate(testPosts):
        if p in ast.literal_eval(users[u]['recommends']):
            matrix[i][j] = 1
            print(i,j)

matrix

0 36
0 1193
0 3998
1 453
1 525
1 1375
1 1638
1 2893
1 3199
2 674
2 889
2 1006
2 1445
2 2242
2 2426
2 2590
2 2991
2 4250
5 2137
6 37
6 364
6 612
6 820
6 1138
6 1563
6 1724
6 1892
6 3098
7 704
7 919
7 1000
7 1153
7 2443
7 2497
7 3098
7 3233
7 3672
7 4002
11 1887
12 157
12 1361
12 1557
12 1918
12 2462
12 2642
12 3054
12 3877
12 4046
12 4204
14 1046
14 3987
16 422
16 813
16 1949
16 1997
16 2625
16 2729
16 2744
16 3117
16 3136
16 4036
17 22
17 485
17 3442
17 3712
17 4071
18 180
18 1234
18 2154
19 177
19 1365
19 1812
19 2164
19 2384
19 2694
19 2855
19 3938
19 4256
20 1853
20 2187
20 2267
20 2587
20 3883
20 3976
21 380
21 785
21 1094
21 1323
21 1828
21 1842
21 2456
21 3002
21 3554
21 4064
22 44
22 122
22 298
22 366
22 631
22 707
22 742
22 1254
22 2311
23 2262
27 27
27 855
27 1016
27 2208
27 2214
27 3254
27 3945
27 3990
27 4118
28 714
28 1909
29 359
29 484
29 730
29 888
29 2553
29 2754
29 3194
29 3326
29 4273
31 917
32 12
32 342
32 1514
32 1568
32 1930
32 2470
32 3613
32 3678
32 4088
32 4214
3

288 3
288 199
288 1624
288 2314
288 2686
288 2828
288 3103
288 4066
289 1093
289 2659
290 738
290 1570
290 1941
290 2283
290 2442
290 2743
290 3546
290 3772
290 4106
290 4283
291 56
291 946
291 1090
291 1291
291 2748
291 4058
292 3680
293 1607
293 1990
293 2148
293 3452
293 3459
293 4013
293 4186
294 75
294 404
294 1131
294 2629
294 2903
294 3332
294 3967
294 4011
294 4247
295 63
295 258
295 519
295 1073
295 1906
295 1946
295 1954
295 3262
295 3868
295 4168
296 273
296 481
296 1047
296 1838
296 1916
296 2526
296 2819
296 3306
296 3532
297 607
297 931
297 1792
297 1923
297 1960
297 2124
297 2363
297 2843
298 922
299 1538
300 243
300 776
300 1904
300 1938
300 2094
300 2374
300 2402
300 2829
300 3955
301 620
301 761
301 1890
301 3863
302 730
303 1899
306 68
306 465
306 3270
306 3949
309 961
309 1606
309 1683
309 3252
309 3462
309 3659
309 4084
309 4105
309 4230
310 492
310 970
310 1227
310 1471
310 2180
310 2735
310 2916
310 2971
310 3466
310 3482
311 441
311 838
311 1800
311 2447
311 246

596 2104
596 2635
596 3119
597 148
597 153
597 823
597 1280
597 1428
597 1450
597 1465
597 2028
597 2235
597 3110
599 79
599 386
599 980
599 2724
600 1356
601 1158
601 2412
601 3652
601 4069
601 4111
602 106
602 1437
602 1672
602 2151
602 3524
605 40
605 1133
605 1687
605 1795
605 2391
605 2427
605 2940
605 3016
606 2
606 1985
606 2478
606 2638
606 2782
606 2959
606 3710
607 1512
607 1628
607 2668
607 2993
607 4084
612 551
612 856
612 2915
612 3328
612 4219
614 1123
614 2251
614 4031
615 686
615 1459
615 1477
615 1841
615 2114
615 2524
615 3279
615 3896
619 301
619 433
619 461
619 800
619 2091
619 3105
619 3653
619 4262
620 384
620 2643
620 2872
620 3889
621 981
621 1312
621 1554
621 2004
621 2126
621 2621
621 2939
621 3001
621 3594
621 4091
624 379
624 1202
624 1591
624 1725
624 2155
624 3218
624 3219
624 3538
624 3745
624 4112
627 2522
629 198
629 355
629 476
629 1100
629 1338
629 1664
629 2019
629 2108
629 2934
629 4061
631 460
631 668
631 781
631 1778
631 3161
631 3227
631 3688
631

837 3656
838 97
838 267
838 341
838 1077
838 1314
838 1506
838 1898
838 1915
838 2820
838 3721
839 803
839 1443
839 3273
840 1248
840 2646
840 3097
840 3839
840 4024
841 129
841 1096
841 1715
841 2127
841 2323
841 3095
841 3187
841 3502
841 3508
841 4094
842 291
842 581
842 778
842 1275
842 2336
842 2603
842 2973
842 3901
844 251
844 1327
844 3894
847 385
847 615
847 1861
847 2319
847 2563
847 2702
847 2949
847 3063
847 3325
848 200
848 265
848 483
848 591
848 656
848 1466
848 1809
848 2149
848 3927
850 710
850 1039
850 1211
850 1253
850 1414
850 1535
850 3281
850 3296
850 4193
852 141
852 206
853 289
853 2774
853 3468
853 3469
853 3952
855 971
858 2999
858 3079
859 526
859 2990
859 3977
860 2682
863 282
863 424
863 1135
863 1462
863 1953
863 2474
863 2827
863 2963
863 3493
864 600
864 1831
864 2250
865 949
865 4268
867 468
867 682
867 1276
867 1345
867 2396
867 2644
867 3209
867 3469
867 3670
867 3858
868 567
868 575
868 799
868 893
868 969
868 1104
868 1620
868 1742
868 2581
869 897


1080 534
1080 1008
1080 1031
1080 1346
1080 3430
1080 4037
1080 4224
1081 3048
1081 3220
1081 3345
1083 958
1083 1212
1083 1251
1083 1346
1083 1385
1083 2736
1083 2824
1083 2910
1083 3929
1083 4161
1085 55
1085 963
1085 3717
1086 312
1086 489
1086 824
1086 1394
1086 1738
1086 2153
1086 2519
1086 3598
1086 3867
1086 4238
1087 596
1087 822
1087 1752
1087 2230
1087 2331
1087 2359
1087 2950
1087 3258
1087 3517
1087 3841
1088 128
1088 324
1088 1292
1088 1369
1088 2569
1088 2920
1088 3050
1088 3345
1091 210
1091 1581
1091 1739
1091 2899
1091 3151
1091 3601
1093 478
1093 1216
1093 2236
1093 2753
1093 4172
1093 4225
1097 276
1097 564
1097 874
1097 1589
1097 2637
1097 2746
1097 2834
1097 3040
1097 3147
1097 3186
1098 1146
1098 1339
1098 1830
1098 3833
1099 413
1099 633
1099 1141
1099 1594
1099 2255
1099 2296
1099 2517
1099 3529
1100 806
1100 1364
1100 1739
1100 2527
1100 2791
1100 4093
1100 4259
1101 572
1101 1080
1101 2475
1101 4026
1101 4180
1102 486
1102 571
1102 1009
1102 1455
1102 2168
110

1320 892
1320 915
1320 1043
1320 1441
1320 1779
1320 1931
1320 2138
1320 2299
1320 4056
1321 904
1321 1164
1321 2719
1321 3464
1321 3488
1321 3819
1321 3897
1323 537
1323 1702
1323 2218
1323 2983
1327 784
1327 1863
1327 2804
1327 3550
1327 4208
1328 211
1328 736
1328 875
1328 1173
1328 2868
1328 3158
1328 3178
1329 101
1329 187
1329 2846
1330 1719
1331 811
1331 1172
1331 3005
1331 3170
1331 3171
1331 3390
1331 3570
1331 3843
1331 3888
1333 665
1333 2006
1333 3616
1334 284
1334 3707
1335 1660
1335 2965
1335 3203
1335 3503
1335 3984
1335 4126
1335 4267
1336 1232
1336 2195
1338 746
1338 1067
1338 1357
1338 2382
1338 2437
1338 3137
1338 3249
1338 3429
1338 3608
1338 3732
1339 1070
1339 1228
1339 3731
1342 222
1342 1225
1342 1427
1342 1852
1342 3232
1342 3280
1342 3946
1342 4033
1343 52
1343 2731
1343 3713
1343 4010
1343 4076
1344 350
1344 444
1344 1508
1344 3301
1344 3861
1344 3971
1346 2776


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [244]:
sum(matrix[0])

3.0

In [253]:
users[testUsers[8]]['recommends']

'[]'

In [138]:
import ast
testPosts = [ast.literal_eval(users[u]['recommends']) for u in testUsers]

In [139]:
testPosts

[['4ccbfbb14314', 'e1b39c8c38ea', '7898ab6bc23e'],
 ['b6ffa82198ee',
  '299fa795d710',
  '5bf9c3b2ec4c',
  'afbed0679311',
  '79104e62e930',
  '5ecd3df24f58'],
 ['ca044cdb7715',
  'db78356063f2',
  '90c75eb7c5b0',
  'c72b1235a564',
  '7e7d2094b77f',
  '3bd8da6f3504',
  '3fe4bf63aa67',
  '98dc3473f085',
  '68a4fa399352'],
 [],
 []]

In [197]:
class MF():

    def __init__(self, R, K, alpha, beta, iterations):
        """
        Perform matrix factorization to predict empty
        entries in a matrix.

        Arguments
        - R (ndarray)   : user-item rating matrix
        - K (int)       : number of latent dimensions
        - alpha (float) : learning rate
        - beta (float)  : regularization parameter
        """

        self.R = R
        self.num_users, self.num_items = R.shape
        self.K = K
        self.alpha = alpha
        self.beta = beta
        self.iterations = iterations

    def train(self):
        # Initialize user and item latent feature matrice
        self.P = np.random.normal(scale=1./self.K, size=(self.num_users, self.K))
        self.Q = np.random.normal(scale=1./self.K, size=(self.num_items, self.K))

        # Initialize the biases
        self.b_u = np.zeros(self.num_users)
        self.b_i = np.zeros(self.num_items)
        self.b = np.mean(self.R[np.where(self.R != 0)])

        # Create a list of training samples
        self.samples = [
            (i, j, self.R[i, j])
            for i in range(self.num_users)
            for j in range(self.num_items)
            if self.R[i, j] > 0
        ]

        # Perform stochastic gradient descent for number of iterations
        training_process = []
        for i in range(self.iterations):
            np.random.shuffle(self.samples)
            self.sgd()
            mse = self.mse()
            training_process.append((i, mse))
            if (i+1) % 10 == 0:
                print("Iteration: %d ; error = %.4f" % (i+1, mse))

        return training_process

    def mse(self):
        """
        A function to compute the total mean square error
        """
        xs, ys = self.R.nonzero()
        predicted = self.full_matrix()
        error = 0
        for x, y in zip(xs, ys):
            error += pow(self.R[x, y] - predicted[x, y], 2)
        return np.sqrt(error)

    def sgd(self):
        """
        Perform stochastic graident descent
        """
        for i, j, r in self.samples:
            # Computer prediction and error
            prediction = self.get_rating(i, j)
            e = (r - prediction)

            # Update biases
            self.b_u[i] += self.alpha * (e - self.beta * self.b_u[i])
            self.b_i[j] += self.alpha * (e - self.beta * self.b_i[j])

            # Update user and item latent feature matrices
            self.P[i, :] += self.alpha * (e * self.Q[j, :] - self.beta * self.P[i,:])
            self.Q[j, :] += self.alpha * (e * self.P[i, :] - self.beta * self.Q[j,:])

    def get_rating(self, i, j):
        """
        Get the predicted rating of user i and item j
        """
        prediction = self.b + self.b_u[i] + self.b_i[j] + self.P[i, :].dot(self.Q[j, :].T)
        return prediction

    def full_matrix(self):
        """
        Computer the full matrix using the resultant biases, P and Q
        """
        return self.b + self.b_u[:,np.newaxis] + self.b_i[np.newaxis:,] + self.P.dot(self.Q.T)

In [224]:
mf = MF(matrix, K=10, alpha=0.1, beta=0.01, iterations=400)

In [225]:
mf

In [226]:
mf.R

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [227]:
mf.train()

Iteration: 10 ; error = 0.5347
Iteration: 20 ; error = 0.2499
Iteration: 30 ; error = 0.1754
Iteration: 40 ; error = 0.1354
Iteration: 50 ; error = 0.1088
Iteration: 60 ; error = 0.0905
Iteration: 70 ; error = 0.0777
Iteration: 80 ; error = 0.0685
Iteration: 90 ; error = 0.0617
Iteration: 100 ; error = 0.0564
Iteration: 110 ; error = 0.0522
Iteration: 120 ; error = 0.0486
Iteration: 130 ; error = 0.0455
Iteration: 140 ; error = 0.0427
Iteration: 150 ; error = 0.0402
Iteration: 160 ; error = 0.0378
Iteration: 170 ; error = 0.0357
Iteration: 180 ; error = 0.0337
Iteration: 190 ; error = 0.0318
Iteration: 200 ; error = 0.0301
Iteration: 210 ; error = 0.0284
Iteration: 220 ; error = 0.0269
Iteration: 230 ; error = 0.0254
Iteration: 240 ; error = 0.0240
Iteration: 250 ; error = 0.0228
Iteration: 260 ; error = 0.0216
Iteration: 270 ; error = 0.0204
Iteration: 280 ; error = 0.0193
Iteration: 290 ; error = 0.0183
Iteration: 300 ; error = 0.0174
Iteration: 310 ; error = 0.0165
Iteration: 320 ; 

[(0, 1.8067780548805943),
 (1, 1.532253026794059),
 (2, 1.317193278794204),
 (3, 1.1399886736392986),
 (4, 0.9913906747407634),
 (5, 0.8676421793324965),
 (6, 0.7620792172217884),
 (7, 0.6736251949011017),
 (8, 0.5982514786094327),
 (9, 0.5346546646724225),
 (10, 0.48094872146614137),
 (11, 0.43576583332019464),
 (12, 0.3974813029930368),
 (13, 0.36498630628510464),
 (14, 0.33761016422316326),
 (15, 0.3143182877941188),
 (16, 0.2944271630202383),
 (17, 0.27728751772042326),
 (18, 0.26283726630720783),
 (19, 0.2499128731136245),
 (20, 0.23848070284121617),
 (21, 0.22869113838657865),
 (22, 0.21980651157138673),
 (23, 0.2118270744100322),
 (24, 0.20451710389550065),
 (25, 0.19785149114303582),
 (26, 0.19155816873667672),
 (27, 0.1859363646663645),
 (28, 0.18048123734272156),
 (29, 0.17540706584665353),
 (30, 0.1704442557041857),
 (31, 0.16595846994794092),
 (32, 0.16153906297890203),
 (33, 0.15721589115230725),
 (34, 0.15331360386122),
 (35, 0.14937049719759263),
 (36, 0.1457125641176429

In [228]:
mf.get_rating(0,0)

0.9891679067539583

In [254]:
matrix[0][:10]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [264]:
for i in range(10):
    for j in range(10):
        print(mf.get_rating(i,j))

0.9891679067539583
0.9989089055208494
1.0028243320502417
0.994882307039628
0.9967341266148212
1.003705498346109
0.9963238779641264
1.0007892060318708
1.0062139725588186
1.0080149570307786
0.9952575771856816
0.9979533137440859
0.9982900863580122
0.9965307048499371
0.9995482266300075
1.000853827699413
0.9979734556100833
0.9969418497407313
1.0001951929495918
0.9983477247638212
0.9979015947373908
1.0021021730859687
0.9992658040062543
1.000058234827789
1.0009410255621667
1.0028554510505627
0.999035359455184
0.9971037273590677
1.0031241947611707
0.9994400972063366
1.0137870224839067
0.9966555571426927
1.012569541771262
1.0142383814822884
1.0334119646987945
1.0224573140565096
1.0135666281950024
0.9818480903033806
0.9817744160221898
1.0071438439949798
1.0306486721847947
0.9899017697373589
0.9814364497648861
1.029746037012573
0.957733987690039
1.0175653584011495
0.9668393063013527
1.030447182631088
0.9616054723642514
1.0111028419239516
0.9840673616938447
1.0068357111889512
1.0047709211688525
1.